<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-documentation/tree/master/ImageCollection/02_image_collection_metadata.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-documentation/blob/master/ImageCollection/02_image_collection_metadata.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/earthengine-py-documentation/master?filepath=ImageCollection/02_image_collection_metadata.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-documentation/blob/master/ImageCollection/02_image_collection_metadata.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# ImageCollection Information and Metadata
As with Images, there are a variety of ways to get information about an `ImageCollection`. The collection can be printed directly to the console, but the console printout is limited to 5000 elements. Collections larger than 5000 images will need to be filtered before printing. Printing a large collection will be correspondingly slower. The following example shows various ways of getting information about image collections programmatically:

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The following script checks if the geehydro package has been installed. If not, it will install geehydro, which automatically install its dependencies, including earthengine-api and folium.

In [1]:
import subprocess

try:
    import geehydro
except ImportError:
    print('geehydro package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geehydro'])

# Import libraries    
import ee
import folium
import geehydro

# Authenticate and initialize Earth Engine API
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [2]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [3]:
# Load a Landsat 8 ImageCollection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 44)) \
    .filter(ee.Filter.eq('WRS_ROW', 34)) \
    .filterDate('2014-03-01', '2014-08-01')
print('Collection: ', collection.getInfo())

Collection:  {'type': 'ImageCollection', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B8', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B9', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B10', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'B11', 'data_type': {'type': 'PixelType', 'precision': 'float'}}, {'id': 'BQA', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}}], 'id': 'LANDSAT/LC08/C01/T1_TOA', 'version': 158246383556185

In [4]:
# Get the number of images.
count = collection.size()
print('Count: ', count.getInfo())

Count:  9


In [5]:
# Get the date range of images in the collection.
range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print('Date range: ', ee.Date(range.get('min')).getInfo(), ee.Date(range.get('max')).getInfo())

Date range:  {'type': 'Date', 'value': 1395168392050} {'type': 'Date', 'value': 1406227557220}


In [6]:
# Change the date format.
range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
start_date = ee.Date(range.get('min')).format("YYYY-MM-dd")
end_date = ee.Date(range.get('max')).format("YYYY-MM-dd")
print('Date range: ', start_date.getInfo(), end_date.getInfo())

Date range:  2014-03-18 2014-07-24


In [7]:
# Get statistics for a property of the images in the collection.
sunStats = collection.aggregate_stats('SUN_ELEVATION')
print('Sun elevation statistics: ', sunStats.getInfo())

Sun elevation statistics:  {'type': 'DataDictionary', 'values': {'max': 67.10252380371094, 'mean': 61.01998392740885, 'min': 46.471065521240234, 'sample_sd': 7.251804209519804, 'sample_var': 52.58866429320915, 'sum': 549.1798553466797, 'sum_sq': 33931.65526085679, 'total_count': 9, 'total_sd': 6.837066576518139, 'total_var': 46.74547937174147, 'valid_count': 9, 'weight_sum': 9, 'weighted_sum': 549.1798553466797}}


In [8]:
# Get statistics for a property of the images in the collection.
clouds = collection.aggregate_array('CLOUD_COVER')
print('Cloud covers: ', clouds.getInfo())

Cloud covers:  [0.05999999865889549, 28.1200008392334, 12.920000076293945, 24.25, 35.439998626708984, 35.709999084472656, 33.029998779296875, 39.97999954223633, 0.3199999928474426]


In [9]:
# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(collection.sort('CLOUD_COVER').first())
print('Least cloudy image: ', image.propertyNames().getInfo())
print('\nCloud cover: ', image.get("CLOUD_COVER").getInfo())

Least cloudy image:  ['system:version', 'system:id', 'RADIANCE_MULT_BAND_5', 'RADIANCE_MULT_BAND_6', 'RADIANCE_MULT_BAND_3', 'RADIANCE_MULT_BAND_4', 'RADIANCE_MULT_BAND_1', 'RADIANCE_MULT_BAND_2', 'K2_CONSTANT_BAND_11', 'K2_CONSTANT_BAND_10', 'system:footprint', 'REFLECTIVE_SAMPLES', 'SUN_AZIMUTH', 'CPF_NAME', 'DATE_ACQUIRED', 'ELLIPSOID', 'google:registration_offset_x', 'google:registration_offset_y', 'STATION_ID', 'RESAMPLING_OPTION', 'ORIENTATION', 'WRS_ROW', 'RADIANCE_MULT_BAND_9', 'TARGET_WRS_ROW', 'RADIANCE_MULT_BAND_7', 'RADIANCE_MULT_BAND_8', 'IMAGE_QUALITY_TIRS', 'TRUNCATION_OLI', 'CLOUD_COVER', 'GEOMETRIC_RMSE_VERIFY', 'COLLECTION_CATEGORY', 'GRID_CELL_SIZE_REFLECTIVE', 'CLOUD_COVER_LAND', 'GEOMETRIC_RMSE_MODEL', 'COLLECTION_NUMBER', 'IMAGE_QUALITY_OLI', 'LANDSAT_SCENE_ID', 'WRS_PATH', 'google:registration_count', 'PANCHROMATIC_SAMPLES', 'PANCHROMATIC_LINES', 'GEOMETRIC_RMSE_MODEL_Y', 'REFLECTIVE_LINES', 'TIRS_STRAY_LIGHT_CORRECTION_SOURCE', 'GEOMETRIC_RMSE_MODEL_X', 'system:

In [10]:
# Limit the collection to the 10 most recent images.
recent = collection.sort('system:time_start', False).limit(10)
images = recent.aggregate_array('system:index').getInfo()
print("Recent images:")
for image in images:
    print(image)

Recent images:
LC08_044034_20140724
LC08_044034_20140708
LC08_044034_20140622
LC08_044034_20140606
LC08_044034_20140521
LC08_044034_20140505
LC08_044034_20140419
LC08_044034_20140403
LC08_044034_20140318


## Display Earth Engine data layers 

In [11]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map